<a href="https://colab.research.google.com/github/maetthil/NASASpaceApps/blob/main/Read_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# # %%capture
# !pip install cdflib
# !pip install pyspedas
# !pip install matplotlib==3.1.2
!pip install pyarrow

IOStream.flush timed out
IOStream.flush timed out
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 2.7 MB/s eta 0:00:0000:0100:01


In [3]:
import glob
import sys
import os
import re
import requests
import numpy as np
import pytz
import datetime
import calendar
import pandas as pd
from tqdm import tqdm

import pyspedas
from pytplot import tplot
import pytplot

import matplotlib.pyplot as plt


## Magnetic Field

In [4]:
def getMagneticField(satellite="wind", trange=['2018-01-01', '2018-01-10']):
    pytplot.del_data()
    if satellite == "wind":
        vars = pyspedas.wind.mfi(trange=trange)
        quants_mag = pytplot.data_quants['BGSE']
        mag = quants_mag.values

        

    else:
        mag = pyspedas.dscovr.mag(trange=trange)
        quants_mag = pytplot.data_quants['dsc_h0_mag_B1GSE']
        mag = quants_mag.values
        mag[mag == -1e+31] = np.nan

    

    time_mag = quants_mag.coords['time'].values
    time_mag = [datetime.datetime.fromtimestamp(i, pytz.timezone("UTC")) for i in time_mag]

    df_mag = pd.DataFrame({"Bx_GSE": mag[:,0], 
                            "By_GSE": mag[:,1],
                            "Bz_GSE": mag[:,2]}, index = time_mag)
    
    return df_mag

## Ion Plasma

In [5]:
def getIonPlasma(satellite="wind", trange=['2018-01-01', '2018-01-10']):
    pytplot.del_data()
    if satellite == "wind":
        vars = pyspedas.wind.swe(trange=trange)
        quants_vGSE = pytplot.data_quants['U_eGSE']
        quants_therSp = pytplot.data_quants['T_elec']
        quants_Np = pytplot.data_quants['N_elec']

        vGSE = quants_vGSE.values
        therSp = quants_therSp.values
        Np = quants_Np.values

        

    else:
        vars = pyspedas.dscovr.fc(trange=trange)

        quants_vGSE = pytplot.data_quants['dsc_h1_fc_V_GSE']
        quants_therSp = pytplot.data_quants['dsc_h1_fc_THERMAL_SPD']
        quants_Np = pytplot.data_quants['dsc_h1_fc_Np']

        vGSE = quants_vGSE.values
        therSp = quants_therSp.values
        Np = quants_Np.values

        vGSE[vGSE == -1e+31] = np.nan
        therSp[therSp == -1e+31] = np.nan
        Np[Np == -1e+31] = np.nan

    

    time_fc = quants_vGSE.coords['time'].values
    time_fc = [datetime.datetime.fromtimestamp(i, pytz.timezone("UTC")) for i in time_fc]

    df_ions = pd.DataFrame({"Vx_GSE": vGSE[:,0], 
                             "Vy_GSE": vGSE[:,1],
                             "Vz_GSE": vGSE[:,2],
                             "Therm_spd": therSp,
                             "Np":Np }, index = time_fc)
    
    return df_ions

In [20]:
ions = getIonPlasma(satellite="dscovr", trange=['2018-01-01', '2018-01-10'])
ions.head()

01-Oct-22 18:45:39: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/dscovr/h1/faraday_cup/2018/
01-Oct-22 18:45:39: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180101_v08.cdf
01-Oct-22 18:45:40: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180102_v08.cdf
01-Oct-22 18:45:40: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180103_v08.cdf
01-Oct-22 18:45:40: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180105_v08.cdf
01-Oct-22 18:45:40: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180106_v08.cdf
01-Oct-22 18:45:40: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180107_v08.cdf
01-Oct-22 18:45:40: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180108_v08.cdf
01-Oct-22 18:45:40: File is current: dscovr_data/h1/faraday_cup/2018/dscovr_h1_fc_20180109_v08.cdf


,Vx_GSE,Vy_GSE,Vz_GSE,Therm_spd,Np
2018-01-01 00:00:00+00:00,-403.007935,-12.879808,-25.402237,39.441380,14.758022
2018-01-01 00:01:00+00:00,-403.619720,-16.166252,-26.591558,38.992432,14.678254
2018-01-01 00:02:00+00:00,-405.910919,-14.688877,-29.482092,36.989517,14.709406
2018-01-01 00:03:00+00:00,-404.753052,-13.926281,-30.157286,36.591244,14.661954
2018-01-01 00:04:00+00:00,-403.666260,-13.622793,-28.274202,36.837086,14.399885


In [ ]:
mag = getMagneticField(satellite="dscovr", trange=['2018-01-01', '2018-01-10'])
mag.head()

In [21]:
def downloadSaveParquet(downloadFunc, sat, yearInterval, monthInterval):
    for yr in tqdm(range(yearInterval[0], yearInterval[1])):
        for mth in range(monthInterval[0], monthInterval[1]):
            lastDay = calendar.monthrange(yr, mth)[1]
            try:
                output = downloadFunc(satellite=sat, trange=[f'{yr:04d}-{mth:02d}-01', f'{yr:04d}-{mth:02d}-{lastDay:02d}'])
                output.to_parquet(f'{yr:04d}-{mth:02d}_{sat}.parquet', engine="pyarrow")
            except Exception as e:
                print("Variavel nao encontrada")



In [23]:
# 18h por instrumento, de cada satélites (para 6 anos) 
downloadSaveParquet(getMagneticField, sat="dscvr", yearInterval=[2017, 2018], monthInterval=[1, 7])

  0%|          | 0/1 [00:00<?, ?it/s]01-Oct-22 18:51:18: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/dscovr/h0/mag/2017/
01-Oct-22 18:51:18: Downloading https://spdf.gsfc.nasa.gov/pub/data/dscovr/h0/mag/2017/dscovr_h0_mag_20170101_v01.cdf to dscovr_data/h0/mag/2017/dscovr_h0_mag_20170101_v01.cdf
01-Oct-22 18:51:21: Download complete: dscovr_data/h0/mag/2017/dscovr_h0_mag_20170101_v01.cdf
01-Oct-22 18:51:21: Downloading https://spdf.gsfc.nasa.gov/pub/data/dscovr/h0/mag/2017/dscovr_h0_mag_20170102_v01.cdf to dscovr_data/h0/mag/2017/dscovr_h0_mag_20170102_v01.cdf
01-Oct-22 18:51:23: Download complete: dscovr_data/h0/mag/2017/dscovr_h0_mag_20170102_v01.cdf
01-Oct-22 18:51:23: Downloading https://spdf.gsfc.nasa.gov/pub/data/dscovr/h0/mag/2017/dscovr_h0_mag_20170103_v01.cdf to dscovr_data/h0/mag/2017/dscovr_h0_mag_20170103_v01.cdf
01-Oct-22 18:51:26: Download complete: dscovr_data/h0/mag/2017/dscovr_h0_mag_20170103_v01.cdf
01-Oct-22 18:51:26: Downloading https://spdf.gsfc.n

In [ ]:
# dados de ions do plasma 18h por instrumento, para cada sat[elite]
downloadSaveParquet(getIonPlasma, sat="dscvr", yearInterval=[2017, 20218], monthInterval=[1, 7])